<a href="https://colab.research.google.com/github/amittpai/GenAI/blob/main/Final_Project_Amit_Pai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project : A Case Study of InnovaTech Solutions**

**Business Overview:**

InnovaTech Solutions, a dynamic and forward-thinking technology company, has made significant strides in the computing industry with a focus on developing high-quality laptops. Established over a decade ago, InnovaTech has gained a reputation for its innovative approach and commitment to customer satisfaction, creating a significant footprint in both physical and online retail spaces.
InnovaTech has expanded its presence in the digital retail world, especially on e-commerce giants like Amazon. This strategic move has not only widened its customer base but also resulted in a large influx of customer feedback, primarily in the form of online reviews. The company's products, notably its range of laptops, have become popular choices on these platforms, leading to an abundance of valuable but underutilized customer data.

**Current Challenge:**

InnovaTech currently analyzes customer reviews using basic sentiment analysis tools, which only provide a superficial understanding of customer opinions. In the competitive landscape of the laptop market, a more detailed and aspect-oriented analysis is crucial. Understanding specific customer sentiments on different aspects of laptops, such as user screen, technical specifications, etc, which is vital for targeted product improvements.

**Objective:**

The primary goal is to conduct a comprehensive aspect-based sentiment analysis of customer reviews for InnovaTech’s laptops, specifically focusing on three critical aspects: the laptop screen, keyboard, and mousepad. These components have been identified as crucial determinants of customer satisfaction and product usability. Project aims to provide nuanced insights into specific areas of customer satisfaction, dissatisfaction, and neutral feedback.The ultimate goal is to enhance overall product quality and customer experience, solidifying InnovaTech's position as a leader in the laptop market.



**Data Description:**

The dataset titled "laptop_reviews.csv" is structured to facilitate aspect-based sentiment analysis for laptop reviews. Here's a brief description of the data columns:

1. id: This column contains unique identifiers for each review entry. It helps in distinguishing and referencing individual reviews
2. text: This column includes the actual text of the laptop reviews. The reviews are likely composed of customer opinions and experiences regarding different aspects of the laptops.
3. aspects:Contains structured information about specific aspects mentioned in each review like 'RAM', 'screen', 'keyboard', 'mousepad', and others relevant to laptop features.
4. category:Provide an additional layer of classification (positive, negative and neutral) for the mentioned aspects.

# 1. Setup

### 1.1 Installation

In [1]:
!pip install openai==0.28.0 tiktoken datasets session-info --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 3.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


### 1.2 Imports

1. Import all Python packages required to access the Azure Open AI API.
2. Import additional packages required to access datasets and create examples.

In [2]:
# Import all Python packages required to access the Azure Open AI API.
# Import additional packages required to access datasets and create examples.

import openai
import json
import random
import tiktoken
import session_info

import pandas as pd
import numpy as np


# Additional packages
from datasets import load_dataset
from collections import Counter
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

### 1.3 Authentication

In [4]:
with open('config.json', 'r') as az_creds:
    data = az_creds.read()

In [5]:
creds = json.loads(data)

In [6]:
openai.api_key = creds["AZURE_OPENAI_KEY"]
openai.api_base = creds["AZURE_OPENAI_ENDPOINT"]
openai.api_type = creds["AZURE_OPENAI_APITYPE"]
openai.api_version = creds["AZURE_OPENAI_APIVERSION"]

In [7]:
chat_model_id = creds["CHATGPT_MODEL"]

In [8]:
chat_model_id

'gen-ai-gpt35-ap'

### 1.4 Utilities

Define token counter to keep track of the completion window available in the prompt.

In [9]:
# Writing a function to keep track of the token count for the LLM model
def num_tokens_from_messages(messages):
  encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

  # initiliaze tokens per message as 3
  tokens_per_message = 3

  # intialize num_tokens to 0
  num_tokens = 0

  # Loop though messages and appened the token count to num_tokens
  for msg in messages:
    num_tokens += tokens_per_message
    for key, value in msg.items():
      num_tokens += len(encoding.encode(value))

  # Make sure to account for the reply message
  num_tokens += 3

  return num_tokens

# Task: Aspect-Based Sentiment Analysis (ABSA)

### Step 1: Define objectives & Metrics

To evaluate model performance, we judge the accuracy of the aspects + sentiment assignnment per aspect.For example, if aspects identified by the LLM do not match the ground truth for a specific input, we count this prediction to be incorrect. A correct prediction is one where all the aspects are correctly idenfied and further the sentiment assignment for each aspect is also correctly identified

In [10]:
# Function to compute accuracy
# Params: gold_examples, model_predictions, ground_truths

def compute_accuracy(gold_examples, model_predictions, ground_truths):
    # Initialize variables to keep track of correct and total predictions
    correct_predictions = 0
    total_predictions = len(gold_examples)

    # Iterate through each prediction and ground truth pair
    for pred, truth in zip(model_predictions, ground_truths):
        if pred == truth:
            correct_predictions += 1

    # Calculate accuracy as the ratio of correct predictions to total predictions
    accuracy = correct_predictions / total_predictions

    return accuracy

### Step 2: Assemble Data

1. Use "laptop_review.csv" dataset.
2. Identify distribution of aspects in examples.
3. Identify distribution of aspects in gold examples.

In [11]:
df = pd.read_csv("laptop_reviews.csv")
df["category"] = df["category"].str.replace("'", '"').str.replace("array(", "", regex=False).str.replace(",dtype=object)", "", regex=False).str.replace(",dtype=int16)", "", regex=False).str.lower()
df["category"] = df["category"].apply(json.loads)

In [12]:
# Test
df.head(4)

,id,text,aspects,category
0,1,The RAM is good. The design is decent.,"{'term':array(['RAM','design'],dtype=object),'...","{'category': ['ram', 'design'], 'polarity': ['..."
1,2,The screen is amazing. The design is impressiv...,"{'term':array(['screen','design','mousepad'],d...","{'category': ['screen', 'design', 'mousepad'],..."
2,3,The GPU is adequate. The camera is average. Th...,"{'term':array(['GPU','camera','software','keyb...","{'category': ['gpu', 'camera', 'software', 'ke..."
3,4,The RAM is terrible. The battery is fair. The ...,"{'term':array(['RAM','battery','design'],dtype...","{'category': ['ram', 'battery', 'design'], 'po..."


In [13]:
# sample the full dataset first
laptop_reviews_examples_df, laptop_reviews_gold_examples_df = train_test_split(
    df, #<- the full dataset
    test_size=0.2, #<- 20% random sample selected for gold examples
    random_state=100 #<- ensures that the splits are the same for every session
)

In [14]:
# Check sampling
laptop_reviews_examples_df.shape

(80, 4)

In [15]:
# Check gold sampling
laptop_reviews_gold_examples_df.shape

(20, 4)

create a lookup index with each of the aspects as keys and a list of all reviews that are annotated with the aspect as the value


In [16]:
examples_aspect_index = {
    'mousepad': [],
    'keyboard':[],
    'screen':[]
}

gold_examples_aspect_index = {
    'mousepad': [],
    'keyboard':[],
    'screen':[]
}

# Distribution of aspects in examples

In [17]:
# loop over the categories and then map the id to the corresponding aspect
for id, category in zip(laptop_reviews_examples_df.id, laptop_reviews_examples_df.category):
  for key in examples_aspect_index.keys():
    if key in category['category']:
      examples_aspect_index[key].append(id)

In [18]:
for key in examples_aspect_index:
  print(f"Number of examples for aspect {key}: {len(examples_aspect_index[key])}")

Number of examples for aspect mousepad: 35
Number of examples for aspect keyboard: 26
Number of examples for aspect screen: 23


# Distribution aspects in gold examples

In [19]:
# loop over the categories and then add the id to the corresponding aspect
for id, category in zip(laptop_reviews_gold_examples_df.id,
                        laptop_reviews_gold_examples_df.category):
  for key in gold_examples_aspect_index.keys():
    if key in category['category']:
      gold_examples_aspect_index[key].append(id)

In [20]:
for key in gold_examples_aspect_index:
  print(f"Number of examples for aspect {key}: {len(gold_examples_aspect_index[key])}")

Number of examples for aspect mousepad: 7
Number of examples for aspect keyboard: 5
Number of examples for aspect screen: 11


# Sample the columns and insert it into gold_examples

In [21]:
# pick columns to sample into the gold_examples
columns_selected = ['id', 'text', 'category']

In [22]:
gold_examples = json.loads((
    laptop_reviews_gold_examples_df.loc[:, columns_selected]
                                    .sample(5, random_state=100)
                                    .to_json(orient='records')
))

In [23]:
# Test
gold_examples[0]

{'id': 6,
 'text': 'The battery is poor. The camera is average. The GPU is fair. The screen is terrible.',
 'category': {'category': ['battery', 'camera', 'gpu', 'screen'],
  'polarity': ['negative', 'neutral', 'neutral', 'negative']}}

### Step 3: Derive Prompt

#### Create prompts

# Generate a User message template

In [24]:
user_message_template = """```{customer_review}```"""

**1. Zero-shot prompt**

In [25]:
zero_shot_system_message = """
You are an AI assisstant who is tasked to perform aspect based sentiment analysis of customer reviews for InnovaTech’s products presented as input delimited by triple backticks ```.
Each review contains following aspects like: hardware, mousepad, design, RAM, keyboard, screen, camera, battery, software, GPU

For each review presented as input:
- Identify if there are any of the 3 aspects (mousepad, keyboard, screen) mentioned in the review.
- Assign a sentiment polarity (positive, negative or neutral) for each aspect

Arrange your response as a JSON object with the following format:
{
  "category":[list of aspects],
  "polarity":[list of corresponding polarities for each aspect]
}
"""

In [26]:
zero_shot_prompt = [{'role':'system', 'content':zero_shot_system_message}]

In [27]:
# get the token value from the prompt
num_tokens_from_messages(zero_shot_prompt)

154

**2.Few-shot prompt**

In [28]:
few_shot_system_message = """
You are an AI assisstant who is tasked to perform aspect based sentiment analysis of customer reviews for InnovaTech’s products presented as input delimited by triple backticks ```.
Each review contains following aspects like: hardware, mousepad, design, RAM, keyboard, screen, camera, battery, software, GPU

For each review presented as input:
- Identify if there are any of the 3 aspects (mousepad, keyboard, screen) mentioned in the review.
- Assign a sentiment polarity (positive, negative or neutral) for each aspect

Arrange your response as a JSON object with the following format:
{
  "category":[list of aspects],
  "polarity":[list of corresponding polarities for each aspect]
}
"""

# Assemble examples to go along with few shot message
# and create a few shot prompt

In [29]:
# Function to create example
# The function returns a JSON list with random examples from an input dataset

def create_examples(datasets, n=4):
  columns_to_select = ['id', 'text', 'category']

  example_ids = []
  # create an aspect index dictionary
  aspect_index = {
    'mousepad': [],
    'keyboard':[],
    'screen':[]
  }
  #iterate through the dataset id and category and map the ids
  for id, category in zip(datasets.id, datasets.category):
    for key in aspect_index.keys():
      if key in category['category']:
        aspect_index[key].append(id)

  # get the distribution aspects
  for key in aspect_index:
    example_ids.extend(np.random.choice(aspect_index[key], n).tolist())

  examples = datasets.loc[datasets.id.isin(example_ids), columns_to_select]

  # return the list as json
  return examples.to_json(orient='records')


In [30]:
# Test
create_examples(laptop_reviews_examples_df)

'[{"id":87,"text":"The software is unpleasant. The keyboard is average. The design is disappointing. The screen is unpleasant.","category":{"category":["software","keyboard","design","screen"],"polarity":["negative","neutral","negative","negative"]}},{"id":74,"text":"The mousepad is average. The GPU is terrible. The battery is terrible. The RAM is fair.","category":{"category":["mousepad","gpu","battery","ram"],"polarity":["neutral","negative","negative","neutral"]}},{"id":45,"text":"The RAM is fair. The GPU is good. The screen is amazing.","category":{"category":["ram","gpu","screen"],"polarity":["neutral","positive","positive"]}},{"id":46,"text":"The design is adequate. The keyboard is bad.","category":{"category":["design","keyboard"],"polarity":["neutral","negative"]}},{"id":20,"text":"The screen is adequate. The keyboard is unpleasant. The hardware is fair. The camera is impressive.","category":{"category":["screen","keyboard","hardware","camera"],"polarity":["neutral","negative",

# Create the few shot prompt

In [31]:
# function that creates prompt and returns it as list of dictionaries in OpenAI format

def create_prompt(system_message, examples, user_message_template):
  # Initiliaze few shot prompt with the system message
  few_shot_prompt = [{'role':'system', 'content': system_message}]

  # Loop through the examples and then append to few_shot_prompt list
  for example in json.loads(examples):
    example_input = example['text']
    example_absa = example['category']
    # append the customer review
    few_shot_prompt.append(
        {
            'role': 'user',
            'content': user_message_template.format(
                customer_review = example_input)
        }
    )
    # append the LLM analysis
    few_shot_prompt.append({'role':'assistant', 'content': f"{example_absa}"}
    )

  return few_shot_prompt

In [32]:
examples = create_examples(laptop_reviews_examples_df)
few_shot_prompt = create_prompt(few_shot_system_message, examples, user_message_template)

In [33]:
# Test
few_shot_prompt

[{'role': 'system',
  'content': '\nYou are an AI assisstant who is tasked to perform aspect based sentiment analysis of customer reviews for InnovaTech’s products presented as input delimited by triple backticks ```.\nEach review contains following aspects like: hardware, mousepad, design, RAM, keyboard, screen, camera, battery, software, GPU\n\nFor each review presented as input:\n- Identify if there are any of the 3 aspects (mousepad, keyboard, screen) mentioned in the review.\n- Assign a sentiment polarity (positive, negative or neutral) for each aspect\n\nArrange your response as a JSON object with the following format:\n{\n  "category":[list of aspects],\n  "polarity":[list of corresponding polarities for each aspect]\n}\n'},
 {'role': 'user',
  'content': '```The hardware is good. The screen is impressive. The mousepad is poor. The design is amazing.```'},
 {'role': 'assistant',
  'content': "{'category': ['hardware', 'screen', 'mousepad', 'design'], 'polarity': ['positive', 'po

In [34]:
# Test
gold_examples

[{'id': 6,
  'text': 'The battery is poor. The camera is average. The GPU is fair. The screen is terrible.',
  'category': {'category': ['battery', 'camera', 'gpu', 'screen'],
   'polarity': ['negative', 'neutral', 'neutral', 'negative']}},
 {'id': 12,
  'text': 'The design is terrible. The mousepad is unpleasant. The RAM is poor.',
  'category': {'category': ['design', 'mousepad', 'ram'],
   'polarity': ['negative', 'negative', 'negative']}},
 {'id': 51,
  'text': 'The camera is terrible. The keyboard is fair. The screen is adequate.',
  'category': {'category': ['camera', 'keyboard', 'screen'],
   'polarity': ['negative', 'neutral', 'neutral']}},
 {'id': 41,
  'text': 'The GPU is bad. The software is terrible. The battery is great. The RAM is poor.',
  'category': {'category': ['gpu', 'software', 'battery', 'ram'],
   'polarity': ['negative', 'negative', 'positive', 'negative']}},
 {'id': 70,
  'text': 'The camera is standard. The GPU is great. The hardware is excellent. The screen i

#### Evaluate prompts

**1. Define Evaluation scorer**

In [46]:
# Function to evaluate the corresponding prompts
def evaluate_prompt(prompt, gold_examples, user_message_template):

  model_predictions, ground_truths = [], []

  for example in gold_examples:
      user_input = [
          {
              'role':'user',
              'content': user_message_template.format(customer_review=example['text'])
          }
      ]
      try:
          response = openai.ChatCompletion.create(
              deployment_id=chat_model_id,
              messages=prompt+user_input,
              temperature=0
          )

          prediction = response['choices'][0]['message']['content'].replace("'", "\"")

          model_predictions.append(json.loads(prediction.strip().lower().replace("\n ", "")))
          #json.loads(prediction.strip().lower())
          ground_truths.append(example['category'])

      except Exception as e:
          print("exception", e)

  accuracy = compute_accuracy(gold_examples, model_predictions, ground_truths)

  return accuracy

**2. Evaluate zero shot prompt**

In [47]:
# Zero Shot
evaluate_prompt(zero_shot_prompt, gold_examples, user_message_template)

1.0

**3. Evaluate few shot prompt**

In [37]:
# Few Shot
evaluate_prompt(few_shot_prompt, gold_examples, user_message_template)

1.0

**4. In summary, compute the average (mean) and measure the variability (standard deviation) of the evaluation scores.**

In [38]:
num_eval_runs = 10

In [39]:
zero_shot_performance = []
few_shot_performance = []

In [48]:
# Zero Shot
for _ in tqdm(range(num_eval_runs)):

    # For each run create a new sample of examples
    examples = create_examples(laptop_reviews_examples_df)

    # Assemble the few shot prompt with these examples
    zero_shot_prompt = create_prompt(zero_shot_system_message, examples, user_message_template)
    # Evaluate prompt accuracy on gold examples
    zero_shot_accuracy = evaluate_prompt(zero_shot_prompt, gold_examples, user_message_template)

    zero_shot_performance.append(zero_shot_accuracy)

100%|██████████| 10/10 [00:20<00:00,  2.07s/it]


In [50]:
# Few Shot
for _ in tqdm(range(num_eval_runs)):

    # For each run create a new sample of examples
    examples = create_examples(laptop_reviews_examples_df)

    # Assemble the few shot prompt with these examples
    few_shot_prompt = create_prompt(few_shot_system_message, examples, user_message_template)
    # Evaluate prompt accuracy on gold examples
    few_shot_accuracy = evaluate_prompt(few_shot_prompt, gold_examples, user_message_template)

    few_shot_performance.append(few_shot_accuracy)

100%|██████████| 10/10 [00:20<00:00,  2.06s/it]


In [49]:
# Zero Shot
np.array(zero_shot_performance).mean(), np.array(zero_shot_performance).std()

(0.9800000000000001, 0.05999999999999999)

In [51]:
# Few shot
np.array(few_shot_performance).mean(), np.array(few_shot_performance).std()

(0.99, 0.04358898943540673)

**----------------------------------------------------------------------------End-----------------------------------------------------------------------------------------**